# Import the packages

In [ ]:
import pipeline as pi
from tifffile import imread,imwrite
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time
from placozoan.func_segmentation import get_holes_mask
from placozoan.base_features import SingleObjectFeatures

# Preprocessing

In [ ]:
start = time.time()

In [ ]:
preprocessed,alg_mask = pi.preprocessing('../data/tricho_8_demo.tif')

# Drift computation

### Computing the drift between each plane

In [ ]:
dx, dy = pi.global_drift(alg_mask)

### Filtering to find the moments the camera moved

In [ ]:
df_drift,res = pi.filtering_drift(dx,dy)

# Segmentation of the preprocessed movie

In [ ]:
mask = pi.segmentation_chanvese(preprocessed)

# Extracting the features

## Organism

In [ ]:
prop = {'properties': ['label',
   'centroid',
   'area',
   'perimeter',
   'eccentricity'],'extra_properties': ['convexity']}

segmentator = SingleObjectFeatures(mask,preprocessed, params=prop)
df_props = segmentator.get_features()

In [ ]:
df_props_correct = pi.clean_results(df_props,df_drift,window_size = 3)

## Wound

In [ ]:
wound = get_holes_mask(mask)
segmentator = SingleObjectFeatures(wound,wound, params=prop)
df_props_wound = segmentator.get_features()

In [ ]:
laser_frame = 0

In [ ]:
#df_props_wound_correct = pi.clean_results(df_props_wound,df_drift)
df_props_wound_correct,time_wound = pi.interpolate_wound(df_props_wound,df_props,laser = laser_frame)

In [ ]:
end = time.time() - start
print(f'The whole pipeline took {end/60:.2f} minutes to complete')

# Plotting results and extracting features

In [ ]:
size_pixel = 0.7692 #2.6 #in microns

## Area over time for organism and wound

In [ ]:
sns.set_context('talk') # better plotting

fig,ax = plt.subplots()
ax2 = ax.twinx()

area_w = np.array((df_props_wound_correct.area.values))
area_o = np.array((df_props_correct.area.values))

sns.lineplot(x=df_props_correct.index.values,y = df_props_correct.area*(size_pixel)*(size_pixel),ax = ax,color='tab:blue')

sns.lineplot(x=df_props_wound_correct.index.values,y = df_props_wound_correct.area*(size_pixel)*(size_pixel),
             ax = ax2,color='tab:orange')


ax.set_ylabel('Trichoplax area $\mu m^2$',color = 'tab:blue')
ax2.set_ylabel('Wound area $\mu m^2$',color='tab:orange')
ax.set_xlabel('Frame')
ax.axvline(laser_frame,0,ax.get_ylim()[1],linestyle='--',color='black',alpha=0.8,label='laser ablation')
ax.legend()

plt.show()

## Ratio area wound/area organism

In [ ]:
sns.lineplot(x = df_props.index.values,y = (area_w/area_o)*100)

# Title of the axes

plt.ylabel(r"$ \frac{wound\,area}{organism\,area}$" + '(%)')
plt.xlabel('Frame')

# Plot a dashed line for the laser

plt.vlines(laser_frame,0,max((area_w/area_o)*100),linestyle='--',color='black',alpha=0.8,label='laser ablation')

plt.legend()
plt.show()

## Area organism

In [ ]:
sns.lineplot(x=df_props_correct.index.values,y = df_props_correct.area*(size_pixel)*(size_pixel),color='tab:blue')

## Trajectory

In [ ]:
sns.set_context('notebook')

fig,ax = plt.subplots()

# scatter plot of the trajectory
sns.scatterplot(x = df_props_correct['centroid-0'],y = df_props_correct['centroid-1'],marker="o"
                ,hue=df_props_correct.index,palette="magma",linewidth=0,ax=ax,hue_norm=(0
                                                                                        ,np.shape(preprocessed)[0])
                ,zorder=1)

#plot the first and last point 

ax.scatter(x = df_props_correct['centroid-0'].values[0],y = df_props_correct['centroid-1'].values[0]
                ,marker="D",linewidth=0,label='begining',zorder=2,s=100)
ax.scatter(x = df_props_correct['centroid-0'].values[-1],y = df_props_correct['centroid-1'].values[-1]
                ,marker="s",linewidth=0,label='end',zorder=2,s=70)

#Plot a dashed line to have the connectivity of the trajectory 

ax.plot(df_props_correct['centroid-0'],df_props_correct['centroid-1'],linestyle='--',zorder=0,alpha=0.8)

# Mise en forme

plt.xlabel('Y-coordinate')
plt.ylabel('X-coordinate')
plt.title('Trajectory of Placozoa movement')
plt.legend(title='Frame')
plt.tight_layout()

# if you want to save the image 

#plt.savefig('../data/results/figures/complete_pipeline/trajectory')

plt.show()

In [ ]:
end = time.time() - start
print(f'The whole pipeline took {end/60:.2f} minutes to complete')

# Saving the images and dataframes

## Saving the preprocessed image

path_preprocessed = '../data/new_movies/results/preprocessed/preprocessed_19102022_16h43_3.tif'
imwrite(path_preprocessed,preprocessed)

## Saving the mask

path_mask = '../data/new_movies/results/mask/preprocesed_19102022_16h43_3_mask.tif'
imwrite(path_mask,mask)

## Saving the dataframes

path_props= '../data/new_movies/results/tables/props_19102022_16h43_3.csv'
path_props_wound = '../data/new_movies/results/tables/props_wound_19102022_16h43_3.csv'
path_drift = '../data/new_movies/results/tables/drift_19102022_16h43_3.csv'

df_props_correct.to_csv(path_props)
df_props_wound_correct.to_csv(path_props_wound)
df_drift.to_csv(path_drift)

# Optional : visualizing the drift

canva,diffx,diffy = pi.create_canva(df_drift,preprocessed)

rescaled = pi.place_img(canva,diffx,diffy,preprocessed,df_drift,res)

## Saving the canva

path_canva = '../data/tricho_8_wound_drift_visualization.tif'
imwrite(path_canva,rescaled,compression='zlib') #adding compression to reduce file size

In [ ]:
print('Done !')